In [4]:
import requests
from requests import get
import pandas as pd
import bs4 as bs
import re
import numpy as np


In [23]:
urls = []
urls.append("https://www.ewg.org/skindeep/search/?utf8=%E2%9C%93&search=hand+cream")

for i in range(2, 71):
    urls.append("https://www.ewg.org/skindeep/search/?page={}&search=hand+cream&utf8=%E2%9C%93".format(i))

In [28]:
len(sites)

636

In [29]:
for m in urls: 
    main_site = m
    main_html = requests.get(main_site)
    #print(html.status_code)
    main_content = main_html.content
    main_soup = bs.BeautifulSoup(main_content,'html.parser')
    product_tile = main_soup.find_all(class_ = "product-tile")
    for p in product_tile :
        sites.append(p.a['href'])

In [30]:
def item_info(t):
    overview = t.find("tbody").find_all("tr", "ingredient-overview-tr")
    wrapper = t.find("tbody").find_all("tr", "ingredient-more-info-wrapper")
    return overview, wrapper


In [31]:
def extract_more_info(more_info) :
    function = []
    concern = []
    for w in more_info :
        td = w.find_all("td")
        has_func = False
        has_conc = False
        for i in range(len(td)) :
            if td[i].get_text() == 'FUNCTION(S)':
                function.append(td[i+1].get_text())
                has_func = True
            if td[i].get_text() == 'CONCERNS':
                concern.append(td[i+1].get_text())
                has_conc = True
        if not has_func:
            function.append('NA')
        if not has_conc:
            concern.append('NA')
    
    return concern,function
                

In [32]:
def extract_name(resultSet):
    names = []
    name = []
    for i in range(len(resultSet)):
        name.append(resultSet[i].find(class_ = "td-ingredient-interior").getText())
    return name


In [33]:
def get_all_info(html):
    content = html.content
    soup = bs.BeautifulSoup(content,'html.parser')
    table = soup.find("table", {"class": "table-ingredient-concerns"})

    ingredient_overview, ingredient_wrapper = item_info(table)

    name = extract_name(ingredient_overview)
    concern, function = extract_more_info(ingredient_wrapper)
#     print("name len: " + str(len(name)) + " conc len: " + str(len(concern)) + " func len: " + str(len(function)))
    return name, concern, function
    

    

In [35]:
name,concern,function = [],[],[]
for site in sites:
    html = requests.get(site)
    if html.status_code != 200:
        continue
    else:
        try:
            name.extend(get_all_info(html)[0])
            concern.extend(get_all_info(html)[1])
            function.extend(get_all_info(html)[2])
        except:
            continue

In [36]:
len(name),len(concern),len(function)

(1524, 1524, 1524)

In [ ]:
ingredient = pd.DataFrame({
    'name': name,
    'concern':concern,
    'function': function
})
ingredient

In [ ]:
ingredient.to_csv(r'/Users/jihoyeom/Desktop/ingredient.csv', index = False, header=True)